In [2]:
# autoreload
%load_ext autoreload
%autoreload 2

In [3]:
import os
import pandas as pd
from tqdm import tqdm

In [4]:
mimic_iv_notes_parent = "/data/wang/junh/datasets/physionet.org/files/mimic-iv-note/2.2/note"
mimic_iv_path = "/data/wang/junh/datasets/physionet.org/files/mimiciv/2.2"

rad_notes_f_path = os.path.join(mimic_iv_notes_parent, "radiology.csv.gz")
rad_notes_df = pd.read_csv(rad_notes_f_path, low_memory=False)
rad_notes_df['charttime'] = pd.to_datetime(rad_notes_df['charttime'])
rad_notes_df['storetime'] = pd.to_datetime(rad_notes_df['storetime'])

icustays_df = pd.read_csv(os.path.join(mimic_iv_path, "icu", "icustays.csv.gz"), low_memory=False)
icustays_df['intime'] = pd.to_datetime(icustays_df['intime'])
icustays_df['outtime'] = pd.to_datetime(icustays_df['outtime'])

admissions_df = pd.read_csv(os.path.join(mimic_iv_path, "hosp", "admissions.csv.gz"), low_memory=False)
admissions_df['admittime'] = pd.to_datetime(admissions_df['admittime'])
admissions_df['dischtime'] = pd.to_datetime(admissions_df['dischtime'])

In [5]:
notes_df = pd.read_csv(os.path.join(mimic_iv_notes_parent, "discharge.csv"), low_memory=False)

In [6]:
notes_df['charttime'] = pd.to_datetime(notes_df['charttime'])
notes_df['storetime'] = pd.to_datetime(notes_df['storetime'])

In [7]:
print(notes_df.columns)
print("Number of notes: ", len(notes_df))

Index(['note_id', 'subject_id', 'hadm_id', 'note_type', 'note_seq',
       'charttime', 'storetime', 'text'],
      dtype='object')
Number of notes:  331793


In [8]:
print(rad_notes_df.columns)
print("Number of rad notes: ", len(rad_notes_df))

Index(['note_id', 'subject_id', 'hadm_id', 'note_type', 'note_seq',
       'charttime', 'storetime', 'text'],
      dtype='object')
Number of rad notes:  2321355


In [11]:
notes_df = pd.read_pickle("/data/wang/junh/datasets/multimodal/preprocessing/clinic_notes_text.pkl")

In [12]:
rad_notes_df = pd.read_pickle("/data/wang/junh/datasets/multimodal/preprocessing/notes_text.pkl")

In [13]:
print(notes_df['subject_id'].nunique())

145914


In [14]:
print(rad_notes_df['subject_id'].nunique())

237427


In [15]:
notes_df = notes_df[(notes_df['subject_id'].notnull())]
notes_df = notes_df[(notes_df['hadm_id'].notnull())]
rad_notes_df = rad_notes_df[(rad_notes_df['subject_id'].notnull())]
rad_notes_df = rad_notes_df[(rad_notes_df['hadm_id'].notnull())]

In [16]:
merge_notes_df = pd.merge(notes_df, rad_notes_df[['subject_id', 'hadm_id', 'text']], on=['subject_id', 'hadm_id'], how='right')

/data/wang/junh/envs/fuse_env/lib/python3.8/site-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


In [17]:
print(merge_notes_df.columns)
print("Number of merged notes: ", len(merge_notes_df))

Index(['note_id', 'subject_id', 'hadm_id', 'note_type', 'note_seq',
       'charttime', 'storetime', 'text_x', 'stay_id', 'icu_time_delta',
       'hosp_time_delta', 'text_y'],
      dtype='object')
Number of merged notes:  832140


In [18]:
merge_notes_df.rename(columns={'text_x': 'clinic_text', 'text_y': 'text'}, inplace=True)

In [22]:
mm_dir = "/data/wang/junh/datasets/multimodal"
output_dir = os.path.join(mm_dir, "preprocessing")

merge_notes_df.to_pickle(os.path.join(output_dir, "merge_notes_text.pkl"))
print(merge_notes_df.shape)

(832140, 12)


In [20]:
merge_notes_df

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,clinic_text,stay_id,icu_time_delta,hosp_time_delta,text
0,10000032-DS-21,10000032,22595853.0,DS,21.0,2180-05-07,2180-05-09 15:26:00,\nName: ___ Unit No: _...,None,None,1.616667,EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ...
1,10000032-DS-21,10000032,22595853.0,DS,21.0,2180-05-07,2180-05-09 15:26:00,\nName: ___ Unit No: _...,None,None,1.616667,"INDICATION: ___ HCV cirrhosis c/b ascites, hi..."
2,10000032-DS-24,10000032,25742920.0,DS,24.0,2180-08-07,2180-08-10 05:43:00,\nName: ___ Unit No: _...,None,None,24.266667,EXAMINATION: PARACENTESIS\n\nINDICATION: ___...
3,10000032-DS-24,10000032,25742920.0,DS,24.0,2180-08-07,2180-08-10 05:43:00,\nName: ___ Unit No: _...,None,None,24.266667,EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ...
4,10000117-DS-22,10000117,27988844.0,DS,22.0,2183-09-21,2183-09-29 16:23:00,\nName: ___ Unit No: ___\n...,None,None,53.833333,EXAMINATION: HIP NAILING IN OR W/FILMS AND FL...
...,...,...,...,...,...,...,...,...,...,...,...,...
832135,19999987-DS-2,19999987,23865745.0,DS,2.0,2145-11-11,2145-11-11 13:13:00,\nName: ___ Unit No: __...,None,None,194.366667,"HISTORY: ___, with left occipital bleeding. ..."
832136,19999987-DS-2,19999987,23865745.0,DS,2.0,2145-11-11,2145-11-11 13:13:00,\nName: ___ Unit No: __...,None,None,194.366667,INDICATION: ___ female intubated for head ble...
832137,19999987-DS-2,19999987,23865745.0,DS,2.0,2145-11-11,2145-11-11 13:13:00,\nName: ___ Unit No: __...,None,None,194.366667,HISTORY: ___ woman with left occipital hemorr...
832138,19999987-DS-2,19999987,23865745.0,DS,2.0,2145-11-11,2145-11-11 13:13:00,\nName: ___ Unit No: __...,None,None,194.366667,PORTABLE CHEST OF ___\n\nCOMPARISON: ___ radi...


In [7]:
rad_notes_df

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text,stay_id,icu_time_delta,hosp_time_delta
0,10000032-RR-14,10000032,None,RR,14,2180-05-06 21:19:00,2180-05-06 23:32:00,EXAMINATION: CHEST (PA AND LAT)\n\nINDICATION...,None,None,None
1,10000032-RR-15,10000032,22595853,RR,15,2180-05-06 23:00:00,2180-05-06 23:26:00,EXAMINATION: LIVER OR GALLBLADDER US (SINGLE ...,None,None,0.616667
2,10000032-RR-16,10000032,22595853,RR,16,2180-05-07 09:55:00,2180-05-07 11:15:00,"INDICATION: ___ HCV cirrhosis c/b ascites, hi...",None,None,11.533333
3,10000032-RR-18,10000032,None,RR,18,2180-06-03 12:46:00,2180-06-03 14:01:00,EXAMINATION: Ultrasound-guided paracentesis.\...,None,None,None
4,10000032-RR-20,10000032,None,RR,20,2180-07-08 13:18:00,2180-07-08 14:15:00,EXAMINATION: Paracentesis\n\nINDICATION: ___...,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
2321350,19999987-RR-17,19999987,23865745,RR,17,2145-11-02 22:37:00,2145-11-03 18:55:00,"HISTORY: ___, with left occipital bleeding. ...",None,None,0.983333
2321351,19999987-RR-18,19999987,23865745,RR,18,2145-11-03 04:35:00,2145-11-03 10:46:00,INDICATION: ___ female intubated for head ble...,36195440,5.6,6.95
2321352,19999987-RR-19,19999987,23865745,RR,19,2145-11-03 16:40:00,2145-11-04 08:36:00,HISTORY: ___ woman with left occipital hemorr...,36195440,17.683333,19.033333
2321353,19999987-RR-20,19999987,23865745,RR,20,2145-11-04 05:10:00,2145-11-04 08:58:00,PORTABLE CHEST OF ___\n\nCOMPARISON: ___ radi...,36195440,30.183333,31.533333


In [13]:
notes_df

,note_id,subject_id,hadm_id,note_type,note_seq,charttime,storetime,text,stay_id,icu_time_delta,hosp_time_delta
0,10000032-DS-21,10000032,22595853,DS,21,2180-05-07,2180-05-09 15:26:00,\nName: ___ Unit No: _...,None,None,1.616667
1,10000032-DS-22,10000032,22841357,DS,22,2180-06-27,2180-07-01 10:15:00,\nName: ___ Unit No: _...,None,None,5.55
2,10000032-DS-23,10000032,29079034,DS,23,2180-07-25,2180-07-25 21:42:00,\nName: ___ Unit No: _...,None,None,35.416667
3,10000032-DS-24,10000032,25742920,DS,24,2180-08-07,2180-08-10 05:43:00,\nName: ___ Unit No: _...,None,None,24.266667
4,10000084-DS-17,10000084,23052089,DS,17,2160-11-25,2160-11-25 15:09:00,\nName: ___ Unit No: __...,None,None,94.066667
...,...,...,...,...,...,...,...,...,...,...,...
331788,19999828-DS-6,19999828,29734428,DS,6,2147-08-04,2147-08-12 15:36:00,\nName: ___ Unit No: ___...,None,None,391.616667
331789,19999828-DS-7,19999828,25744818,DS,7,2149-01-18,2149-01-19 07:03:00,\nName: ___ Unit No: ___...,None,None,223.266667
331790,19999840-DS-20,19999840,26071774,DS,20,2164-07-28,2164-07-29 14:52:00,\nName: ___ Unit No: ___\...,None,None,71.55
331791,19999840-DS-21,19999840,21033226,DS,21,2164-09-17,2164-09-18 01:36:00,\nName: ___ Unit No: ___\...,38978960,110.558889,154.216667


In [14]:
notes_df['stay_id'].notna().sum()

11455